#### Infill NOTEBOOK

This notebook validates various elementary bricks related to Bayesian and non-Bayesian adaptive infill strategies:

1. Expected improvement
2. Lower Confidence Bound
3. max-min Euclidean Distance
4. the generalisation of SMT's nested LHS to arbitrary DOE sizes

**Notes**: the tests are performed for single-fidelity single-objective optimization and adapted from SMT's documentation (see [EGO](https://smt.readthedocs.io/en/latest/_src_docs/applications/ego.html)).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cdist
from scipy.stats import qmc

from aero_optim.mf_sm.mf_models import get_model
from aero_optim.mf_sm.mf_infill import minimize_LCB, maximize_EI, maximize_ED

Forester function and initial DOE

In [ ]:
def forrester(x):
    return (6 * x -2)**2 * np.sin(12 * x - 4)

x_t = np.linspace(0, 1, 3).reshape(-1, 1)
y_t = forrester(x_t)

x_plot = np.linspace(0, 1, 200).reshape(-1, 1)
y_plot = forrester(x_plot)

In [ ]:
fig, ax = plt.subplots()
ax.plot(x_plot, y_plot, color="k", label='exact solution')
ax.scatter(x_t, y_t, marker='o', facecolors="none", edgecolors="blue", label='DOE')
ax.set(xlabel='x', ylabel='y')
ax.legend()
plt.show()

#### 1. LCB

LCB infill input variables are:

- `seed` the random seed
- `niter` the number of LCB iterations
- `(x_t, y_t)` the initial DOE

**Note**: `model` is a single-fidelity kriging surrogate

In [ ]:
seed = 123
n_iter = 12

model = get_model(model_name="smt", dim=1, config_dict={}, outdir="", seed=seed)
model.set_DOE(x_lf=x_t, y_lf=y_t, x_hf=x_t, y_hf=y_t)
model.train()

LCB-based adaptive infill loop

In [ ]:
for _ in range(n_iter):
    new_x = minimize_LCB(model, n_var=1, bound=[0, 1], seed=0, n_gen=10)
    new_y = forrester(new_x)
    print(f"iter {_}, new x {new_x}, new_y {new_y}")
    model.set_DOE(x_lf=x_t, y_lf=y_t, x_hf=new_x, y_hf=new_y)
    model.train()

LCB results are plotted

In [ ]:
fig, ax = plt.subplots()
ax.plot(x_plot, y_plot, color="k", label='objective')
ax.plot(x_plot, model.evaluate(x_plot), color="r", linestyle="dashed", label='model')
ax.fill_between(
    np.ravel(x_plot),
    np.ravel(model.evaluate(x_plot) - 3 * model.evaluate_std(x_plot)),
    np.ravel(model.evaluate(x_plot) + 3 * model.evaluate_std(x_plot)),
    color="lightgrey",
    label="confidence interval"
)
ax.scatter(model.x_hf_DOE, model.y_hf_DOE, marker='o', facecolors="none", edgecolors="blue", label='DOE')
ax.set(xlabel='x', ylabel='y')
ax.legend()
plt.show()

#### 2. EI

EI infill input variables are:

- `seed` the random seed
- `niter` the number of EI iterations
- `(x_t, y_t)` the initial DOE

**Note**: `model` is a single-fidelity kriging surrogate

In [ ]:
n_iter = 8
model = get_model(model_name="smt", dim=1, config_dict={}, outdir="", seed=seed)
model.set_DOE(x_lf=x_t, y_lf=y_t, x_hf=x_t, y_hf=y_t)
model.train()

EI-based adaptive infill loop

In [ ]:
for _ in range(n_iter):
    new_x = maximize_EI(model=model, n_var=1, bound=[0, 1], seed=0, n_gen=10)
    new_y = forrester(new_x)
    print(f"iter {_}, new x {new_x}, new_y {new_y}")
    model.set_DOE(x_lf=new_x, y_lf=new_y, x_hf=new_x, y_hf=new_y)
    model.train()

EI results are plotted

In [ ]:
fig, ax = plt.subplots()
ax.plot(x_plot, y_plot, color="k", label='objective')
ax.plot(x_plot, model.evaluate(x_plot), color="r", linestyle="dashed", label='model')
ax.fill_between(
    np.ravel(x_plot),
    np.ravel(model.evaluate(x_plot) - 3 * model.evaluate_std(x_plot)),
    np.ravel(model.evaluate(x_plot) + 3 * model.evaluate_std(x_plot)),
    color="lightgrey",
    label="confidence interval"
)
ax.scatter(model.x_lf_DOE, model.y_lf_DOE, marker='o', facecolors="none", edgecolors="blue", label='DOE')
ax.set(xlabel='x', ylabel='y')
ax.legend()
plt.show()

#### 3. max-min ED

max-min ED infill input variables are:

- `seed` the random seed
- `niter` the number of EI iterations
- `(x_t, y_t)` the initial DOE

**Note**: `model` is a single-fidelity kriging surrogate

In [ ]:
n_iter = 15
model = get_model(model_name="smt", dim=1, config_dict={}, outdir="", seed=seed)
model.set_DOE(x_lf=x_t, y_lf=y_t, x_hf=x_t, y_hf=y_t)
model.train()

max-min ED-based adaptive infill loop

In [ ]:
for _ in range(n_iter):
    new_x = maximize_ED(DOE=model.get_DOE(), n_var=1, bound=[0, 1], seed=0, n_gen=10)
    new_y = forrester(new_x)
    print(f"iter {_}, new x {new_x}, new_y {new_y}")
    model.set_DOE(x_lf=new_x, y_lf=new_y, x_hf=new_x, y_hf=new_y)
    model.train()

max-min ED results are plotted

In [ ]:
fig, ax = plt.subplots()
ax.plot(x_plot, y_plot, color="k", label='objective')
ax.plot(x_plot, model.evaluate(x_plot), color="r", linestyle="dashed", label='model')
ax.fill_between(
    np.ravel(x_plot),
    np.ravel(model.evaluate(x_plot) - 3 * model.evaluate_std(x_plot)),
    np.ravel(model.evaluate(x_plot) + 3 * model.evaluate_std(x_plot)),
    color="lightgrey",
    label="confidence interval"
)
ax.scatter(model.x_lf_DOE, model.y_lf_DOE, marker='o', facecolors="none", edgecolors="blue", label='DOE')
ax.set(xlabel='x', ylabel='y')
ax.legend()
plt.show()

#### 4. nested LHS

The LHS nested sampling input variables are:

- `seed` the random seed
- `n_lf` the number of low-fidelity samples to draw
- `n_hf` the number of high-fidelity samples to draw

In [ ]:
n_lf = 10
n_hf = 3

Builds the 2D LHS sampler

In [ ]:
sampler = qmc.LatinHypercube(d=2, seed=seed)

In [ ]:
hf_sample = sampler.random(n=n_hf)
x_hf = qmc.scale(hf_sample, *[0, 1])

lf_sample = sampler.random(n=n_lf)
x_lf = qmc.scale(lf_sample, *[0, 1])

Original low- and high-fidelity DOEs

In [ ]:
fig, ax = plt.subplots()
ax.scatter(
    x_lf[:, 0], x_lf[:, 1],
    marker='x', color="red", label='x_lf'
)
ax.scatter(
    x_hf[:, 0], x_hf[:, 1],
    marker='o', facecolors="none", edgecolors="blue", label='x_hf'
)
ax.set(xlabel='x', ylabel='y')
ax.legend()
plt.show()

LHS nearest neighbours extension (see [SMT sources](https://github.com/SMTorg/smt/blob/master/smt/applications/mfk.py#L73-L143))

In [ ]:
# nearest neighbours deletion
ind = []
d = cdist(x_hf, x_lf, "euclidean")
for j in range(x_hf.shape[0]):
    dj = np.sort(d[j, :])
    k = dj[0]
    ll = (np.where(d[j, :] == k))[0][0]
    m = 0
    while ll in ind:
        m = m + 1
        k = dj[m]
        ll = (np.where(d[j, :] == k))[0][0]
    ind.append(ll)

x_lf_nested = np.delete(x_lf, ind, axis=0)
x_lf_nested = np.vstack((x_lf_nested, x_hf))

Nested low- and high-fidelity DOEs

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.scatter(
    x_hf[:, 0], x_hf[:, 1], marker='o', s=40, facecolor="none", color="blue", label='x_hf'
)
ax.scatter(
    x_lf[:, 0], x_lf[:, 1], marker='x', s=40, color="red", label='original x_lf'
)
ax.scatter(
    x_lf_nested[:, 0], x_lf_nested[:, 1], s=20, marker='^', color="k", label='nested x_lf'
)
ax.set(xlabel='x', ylabel='y')
ax.legend()
plt.show()